In [1]:
import os
import numpy as np
import torch
from torch import nn
import pandas as pd
import whisper
import torchaudio
import torchaudio.transforms as at
from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
def dataset_path_list(path, gender):
    file_path_list = list()
    for env in os.listdir(path):
        env_path = path+'\\'+env
        for gender_folder in os.listdir(env_path):
            gender_folder_path = env_path +'\\'+gender
            for gender_file in os.listdir(gender_folder_path):
                gender_file_path = gender_folder_path+'\\'+gender_file
                file_path_list.append(gender_file_path)
    return file_path_list

In [ ]:
# import os
# import glob
# dataset_dir = "C:\\Users\\USER\\Desktop\\kspeech\\dataset\\freeconversation_common\\Validation\\wave_and_txt\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음"
# transcripts_path_list = list(glob.glob(dataset_dir+"\\*.txt"))
# audio_path_list = list(glob.glob(dataset_dir+"\\*.wav"))
# print(len(transcripts_path_list))
# print(len(audio_path_list))

import os
import glob

transcript_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\transcript"
audio_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\audio"

female_transcript_path_list = dataset_path_list(transcript_path, "Female")
female_audio_path_list = dataset_path_list(audio_path, "Female")
print(len(female_transcript_path_list))
print(len(female_audio_path_list))

male_transcript_path_list = dataset_path_list(transcript_path, "Male")
male_audio_path_list = dataset_path_list(audio_path, "Male")
print(len(male_transcript_path_list))
print(len(male_audio_path_list))

whole_transcript_path_list = list()
whole_transcript_path_list.extend(female_transcript_path_list)
whole_transcript_path_list.extend(male_transcript_path_list)
whole_audio_path_list = list()
whole_audio_path_list.extend(female_audio_path_list)
whole_audio_path_list.extend(male_audio_path_list)
print(len(whole_transcript_path_list))
print(len(whole_audio_path_list))

In [ ]:
import random
# print(len(whole_audio_path_list)): 4557966
random.shuffle(whole_audio_path_list)

In [ ]:
#30만개 랜덤으로 골라냄.
train_whole_audio_path_list, test_whole_audio_path_list = whole_audio_path_list[:210000], whole_audio_path_list[210000:300000]

In [ ]:
train_whole_transcript_list = list()
for item in train_whole_audio_path_list:
    start = item.find("o\\")
    end = item.rfind(".wav")
    file_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\transcript\\"+item[start+2:end]+".json.txt"
    if Path(file_path).exists() == True:
        train_whole_transcript_list.append(file_path)
    else: 
        train_whole_audio_path_list.remove(item)
    
test_whole_transcript_list = list()
for item in test_whole_audio_path_list:
    start = item.find("o\\")
    end = item.rfind(".wav")
    file_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\transcript\\"+item[start+2:end]+".json.txt"
    if Path(file_path).exists() == True:
        test_whole_transcript_list.append(file_path)
    else:
        test_whole_audio_path_list.remove(item)

In [ ]:
print(len(train_whole_audio_path_list))
print(len(test_whole_audio_path_list))
print(len(train_whole_transcript_list))
print(len(test_whole_transcript_list))

In [ ]:
#train dataset
train_female_audio_list = list()
train_male_audio_list = list()

for item in train_whole_audio_path_list:
    if item.find('_F')>=0:
        train_female_audio_list.append(item)
    elif item.find('_M')>=0:
        train_male_audio_list.append(item)
    else:
        print("ERROR!", item)

train_female_transcript_list = list()
train_male_transcript_list = list()

for item in train_whole_transcript_list:
    if item.find('_F')>=0:
        train_female_transcript_list.append(item)
    elif item.find('_M')>=0:
        train_male_transcript_list.append(item)
    else:
        print("ERROR!", item)

In [ ]:
train_whole_df = pd.DataFrame({'audio_path':list(), 'transcript_path':list(), 'gender':list()})
for item in train_whole_audio_path_list:
    start = item.find("o\\")
    end = item.rfind(".wav")
    transcript_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\transcript\\"+item[start+2:end]+".json.txt"
    audio_path = item
    if item.find('_F')>=0:
        gender = 'F'
    elif item.find('_M')>=0:
        gender = 'M'
    else:
        print("ERROR!", item)
    tmp_df = pd.DataFrame({'audio_path':[audio_path], 'transcript_path':[transcript_path], 'gender':[gender]})
    train_whole_df = pd.concat([train_whole_df, tmp_df], axis=0, ignore_index=True)

In [ ]:
len(train_whole_df)

In [ ]:
test_whole_df = pd.DataFrame({'audio_path':list(), 'transcript_path':list(), 'gender':list()})
for item in test_whole_audio_path_list:
    start = item.find("o\\")
    end = item.rfind(".wav")
    transcript_path = "D:\\ksponspeech\\자유대화 음성(일반남녀)\\Training\\transcript\\"+item[start+2:end]+".json.txt"
    audio_path = item
    if item.find('_F')>=0:
        gender = 'F'
    elif item.find('_M')>=0:
        gender = 'M'
    else:
        print("ERROR!", item)
    tmp_df = pd.DataFrame({'audio_path':[audio_path], 'transcript_path':[transcript_path], 'gender':[gender]})
    test_whole_df = pd.concat([test_whole_df, tmp_df], axis=0, ignore_index=True)

In [ ]:
test_whole_df.head(5)

In [ ]:
train_whole_df.to_csv("C:\\Users\\USER\\Desktop\\kspeech\\Whisper\\train_whole_df.csv")
test_whole_df.to_csv("C:\\Users\\USER\\Desktop\\kspeech\\Whisper\\test_whole_df.csv")

In [ ]:
print(len(train_female_audio_list))
print(len(train_male_audio_list))
print(len(train_female_transcript_list))
print(len(train_male_transcript_list))